**Identificação do aluno**

**Email:**

**Matrícula:**

# **Análise de Sentimentos em Reviews do IMDb**

O principal objetivo desta tarefa é aplicar três modelos de aprendizado de máquina distintos - Regressão Logística, Naive Bayes e KNN - para realizar a análise de sentimento em um conjunto de dados de *reviews* de usuários sobre filmes no IMDb. Neste laboratório vamos aprender analisar diferentes valores dos parâmetros dos modelos, selecionando aqueles que tiveram melhor desempenho na validação cruzada. Além disso, vamos calcular as principais métricas utilizadas nesse tipo de tarefa e analisar o desempenho dos nossos modelos. Ao final desta tarefa, você deverá ter uma compreensão mais profunda de como esses modelos funcionam, suas vantagens e limitações quando aplicados a dados textuais do mundo real. Este conjunto de dados inclui avaliações de texto juntamente com rótulos de sentimento correspondentes (positivo ou negativo) para a aprendizagem supervisionada.

## **Bibliotecas**

In [ ]:
# Para a leitura e armazenamento estruturado dos dados
import pandas as pd

# Manipulação de texto
import nltk
from nltk                             import SnowballStemmer
from nltk.tokenize                    import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

# Prepraração dos dados
import numpy as np
from sklearn.feature_extraction.text  import CountVectorizer
from sklearn.model_selection          import train_test_split
from sklearn.preprocessing            import StandardScaler

# Modelos de classificação
from sklearn.linear_model import SGDClassifier            #Stocastic Gradient Descent
from sklearn.naive_bayes import MultinomialNB             #Multinominal Naive Bayes
from sklearn.neighbors import KNeighborsClassifier        #K-nearest Neihgbors

# Avaliação
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


## **Leitura dos dados**

Neste Lab, utilizamos um conjunto de dados com avaliações de filmes do IMDb, onde cada review é rotulado como positivo ou negativo. O objetivo é realizar uma análise de sentimentos, iniciando com a leitura dos dados que serão usados para treinar e avaliar modelos de classificação.


In [ ]:
df = pd.read_csv('https://gist.githubusercontent.com/issilva5/44c9406a85b0fed0d62668752cc31b09/raw/49e01d2e8011bdd83d0bc835a518e398ae319303/movie_reviews.csv')
df = df.groupby('sentiment', group_keys=False).sample(n=2500, random_state=42)
df.head(8)

,content,sentiment
3470,this movie joshua is extremely disturbing and ...,neg
23859,this dvd appears to be targetted at someone wh...,neg
17858,william hurt scuba diving scientist us agents ...,neg
9965,this long winded film turns out to be less abo...,neg
20220,not only was he invariably annoying to listen ...,neg
5455,the lives of the saints starts off with an atm...,neg
5656,there are many different versions of this one ...,neg
11157,every once in a long while a movie will come a...,neg


Os dados estão distribuídos em duas classes:

In [ ]:
df.groupby('sentiment').count()

,content
sentiment,
neg,2500
pos,2500


A seguir criaremos uma lista com o texto.

In [ ]:
corpus = df['content'].tolist()
print(corpus[0])

this movie joshua is extremely disturbing and downright pointless it actually makes me shudder to think there are people who would enjoy watching it without giving away the story it is about a young boys reaction to his newborn sister and that is just the tip of the iceberg during the entirety of this movie the viewer is subjected to some of the most unsettling child behavior imaginable adding insult to injury by the end of this movie there is absolutely no real outcome except the fruition of pure evil at the hands of a child no less who outsmarted a whole group of dumb adults there is no redemption no justice served and a whole group of adults who are not smart enough to see what is going on around them frankly i did not enjoy watching this movie it was extremely unsettling even for those who might enjoy horror movies this movie could be too much despite the fact this movie was well acted the story itself is so disturbing that watching it was equivalent to a minute wait in a dentists 

## **Limpeza dos dados**

Antes de aplicar modelos de aprendizado de máquina, você precisará limpar e pré-processar os dados textuais. É esperado que você aplique pelo menos as seguintes tarefas de limpeza dos dados:

- Tokenização
- Remoção de palavras muito pequenas (<= 2) ou muito grandes (>= 15).
- Remoção de stopwords.
- Stemming.

In [ ]:
def remove_palavras_pequenas_grandes(tokens):
  # Implemente uma função que recebe uma lista de tokens
  # e retorna uma lista sem os tokens muito pequenos (< 2)
  # ou muito grandes (> 15).
  pass

def remove_stopwords(tokens):
  # Implemente uma função que recebe uma lista de tokens
  # e retorna uma lista sem stopwords.
  # Dica: use uma lista ou conjunto de stopwords e filtre os tokens com list comprehension.
  pass

def stemming(tokens, stemmer):
  # Implemente uma função que recebe uma lista de tokens
  # e retorna uma lista com os tokens stemmizados.
  # Dica: use um stemmer como o PorterStemmer da biblioteca NLTK.
  pass

In [ ]:
def process_corpus(corpus):
  param_stemmer = SnowballStemmer('english')
  corpus_processed = []
  for document in corpus:
    tokens = word_tokenize(document)
    tokens = remove_palavras_pequenas_grandes(tokens)
    tokens = remove_stopwords(tokens)
    tokens = stemming(tokens, param_stemmer)
    corpus_processed.append(" ".join(tokens))
  return corpus_processed

corpus_processed = process_corpus(corpus)
print(corpus_processed[0])

## **Preparando os dados para os modelos**

1. **Vetorização dos dados**

Primeiramente, realize a vetorização dos dados. Lembre que o classificador naive bayes trabalha melhor com uma vetorização bag-of-words. Regressão logística e KNN podem trabalhar com outros tipos de vetorização ou podem necessitar de alguma normalização ou padronização. Porém, neste laboratório vamos utilizar a vetorização bag-of-words e padronizar os dados utilizando a z-score. Siga os passos abaixo:

In [ ]:
# Você deve instanciar um vetorizador e aplicá-lo ao corpus processado. O naive bayes pressupõe frequências inteiras. Por esse motivo, devemos usar
# um vetorizador binário. Dica: Use a classe CountVectorizer() para uma vetorização binária com contagens e armazene a vetorização na variável X.


#Visualizando parte dos dados binarizados
print(X.toarray()[0][:10])
print(y[0])

2. **Separação dos dados em conjunto de treinamento e teste**

Agora, realize a partição treino e teste dos dados considerando o vetor bag-of-words.

In [ ]:
# Você deve realizar a partição treino e teste dos dados aqui. Dica: Use train_test_split() passando os vetores X, y da célula anterior e escolhendo a porcentagem de separação.
# Utilize 70% dos dados para treinamento. Isso pode ser feito através do parâmetro train_size
# Utilize também os parâmetros random_state=42 e stratify=y

In [ ]:
# Verificando se estamos mantendo o balanceamento no conjunto de treino e teste
classes, counts = np.unique(y_train, return_counts=True)

print('Dados de treino')
for label, count in zip(classes, counts):
    print(f"{label}: {count}")

In [ ]:
# Verificando se estamos mantendo o balanceamento no conjunto de treino e teste
classes, counts = np.unique(y_test, return_counts=True)

print('Dados de teste')
for label, count in zip(classes, counts):
    print(f"{label}: {count}")

In [ ]:
# Como já temos definidos quais são nossos dados de treinamento e teste, podemos criar as versões padronizadas desses dados para os algoritmos de
# regressão logísticia e KNN. Para realizar a padronização, você pode utilizar a classe StandardScaler() e o método fit_transform() passando seus vetores
# Faça a padronização para X_train e X_test armazenando cada resultado em uma nova variável (X_train_pad e X_test_pad)


print(X_train_pad[0][:10])
print(X_test_pad[0][:10])


## **Treinando modelos**

## **Selecionando Parâmetros**


Nossos modelos possuem parâmetros. O naivebayes possui um parâmetro alpha que é relacionado a suavização de laplace; o KNN possui o K que indica quantos vizinhos devemos considerar como sendo os mais próximos; e a regressão logística apresenta o learning rate que diz se devemos ter passos mais apressados ou não para reduzir ou aumentar os valores dos pesos da função. Nesta estapa, vamos utilizar a validação cruzada para encontrar esses parâmetros

1. **Parâmetro alpha do Naive Bayes:**

Vamos tentar encontrar o melhor parâmetro de suavização considerando `[0.5, 1, 1.5, 2, 2.5]`. Para isso, adapte a validação cruzada do código abaixo para iterar sobre esses valores e armazenar a métrica de f-score.

In [ ]:
import sklearn.model_selection as cv

#n_splits indica a quantidade de splits que faremos
cv = cv.StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

alphas = [0.5, 1, 1.5, 2, 2.5]

for alpha in alphas:

    metricas = []  # armazenar as métricas de cada fold

    # Itera sobre cada fold
    for train_idx, test_idx in cv.split(X_train, y_train):

        # Separa os ids de treino e os de teste
        X_train_cv, X_test_cv = X[train_idx], X[test_idx]
        y_train_cv, y_test_cv = y[train_idx], y[test_idx]

        # Nesse ponto instancie o modelo naive bayes para a variável modelo.
        # Ele pode ser instanciando chamando a classe MultinomialNB() e passando o parâmetro alpha=seu_valor alfa atual

        # Para treinar o modelo, utilize a variável model criada acima e chame o método .fit() passando os valores X e y de treino do fold atual

        # Agora, vamos realizar as predições com nosso modelo e armazená-la na variável y_pred.
        # Para isso, use a variável model e chame o método .predict() passando para ela os dados X de teste do fold

        # Calculando a métrica e armazenando-a
        metrica_atual = f1_score(y_test_cv, y_pred, average='micro', labels=['pos',' neg'])
        metricas.append(metrica_atual)

    print(f"Alpha = {alpha:.1f} → Valor médio da métrica: {np.mean(metricas):.4f}")

2. **Parâmetro learning_rate da Regressão Logística**

Vamos tentar encontrar o melhor parâmetro de learning rate para a regressão logística considerando os valores `[0.0001, 0.001, 0.01, 0.1]`. Para isso, adapte a validação cruzada do código abaixo para iterar sobre esses valores e armazenar a métrica de f-score.

In [ ]:
import sklearn.model_selection as cv

#n_splits indica a quantidade de splits que faremos
cv = cv.StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

learning_rates = [0.0001, 0.001, 0.01, 0.1]

for lr in learning_rates:

    metricas = []  # armazenar métricas de cada fold

    # Itera sobre cada fold mas agora estamos usando os vetores
    for train_idx, test_idx in cv.split(X_train_pad, y_train):

        # Separa os ids de treino e os de teste
        X_train_cv, X_test_cv = X[train_idx], X[test_idx]
        y_train_cv, y_test_cv = y[train_idx], y[test_idx]

        # Nesse ponto instancie o modelo de regressão logística para a variável modelo.
        # Ele pode ser instanciando chamando a classe SGDClassifier() e passando o parâmetro eta0=seu_valor lr atual.
        # Use os parâmetros learning_rate = 'constant' e também o random_state=42 para evitar aleatoriedade na inicialização dos pesos da função


        # Treine o modelo e faça as predições nos mesmos moldes da validação cruzada da etapa do Naive Bayes

        #Calculando a métrica e armazenando-a
        metrica_atual = f1_score(y_test_cv, y_pred, average='micro', labels = ['pos', 'neg'])
        metricas.append(metrica_atual)

    print(f"Learning Rate = {lr:.4f} → Valor médio da métrica: {np.mean(metricas):.4f}")

3. **Parâmetro K-vizinhos do KNN**

 Vamos tentar encontrar o melhor parâmetro vizinhança para o KNN. para isso considere `K=[2, 3, 4, 5, 6, 7]`. Adapte a validação cruzada do código abaixo para iterar sobre esses valores e armazenar a métrica de f-score.

In [ ]:
import sklearn.model_selection as cv

#n_splits indica a quantidade de splits que faremos
cv = cv.StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

K=[2, 3, 4, 5, 6, 7]

for k in K:
    metricas = []  # armazenar acurácias de cada fold

    #Itera sobre cada fold mas agora estamos usando os vetores
    for train_idx, test_idx in cv.split(X_train_pad, y_train):

        #Separa os ids de treino e os de teste
        X_train_cv, X_test_cv = X[train_idx], X[test_idx]
        y_train_cv, y_test_cv = y[train_idx], y[test_idx]

        #Nesse ponto instancie o modelo NKN e salve na variavel model.
        #Ele pode ser instanciando chamando a classe KNeighborsClassifier() e passando o parâmetro n_neighbors= seu valor k atual.

        # Treine o modelo e faça as predições nos mesmos moldes da validação cruzada da etapa do Naive Bayes

        #Calculando a métrica e armazenando-a
        metrica_atual = f1_score(y_test_cv, y_pred, average='micro', labels=['pos', 'neg'])
        metricas.append(metrica_atual)

    print(f"K = {k} → Valor médio da métrica: {np.mean(metricas):.4f}")

### **Perguntas**

Responda justificando

1. Qual modelo demonstra superioridade no treinamento? Qual a sua intuição sobre essa superioridade?
- Resposta:

2. Ao aplicarmos a suavização de laplace no modelo de Naive Bayes, os resultados pioram quando aumentamos o alfa? Por que isso acontece?
- Resposta:

4. **Treinando nossos modelos com os melhores parâmetros**

Agora que já sabemos o melhor valor de suavização (NaiveBayes), de learning rate (Regressão Logística), e da quantidade de vizinhos (KNN), vamos treinar nossos modelos com todo o conjunto de treinamento (X_train) e testá-lo no conjunto de teste que estava separado (X_test).

In [ ]:
# Você deve instanciar e treinar os três modelos.
# Dica: Use as classes que você usou SGDClassifier(), MultinomialNB() e KNeighborsClassifier(), passando o parâmetro selecionado conforme a validação cruzada
# Lembre-se que vamos usar os dados vetorizados sem padronização para o Naive Bayes e os com padronização para os demais
# Salve o modelo naive bayes na variável cnb, a regressão linear na variável clr e o KNN na variável knn


## **Avaliação dos modelos**

1. **Predições para cada modelo**

Você deve realizar as predições para cada um dos três modelos. Para cada variável que guarda os objetos dos modelos gerados, use a predict(X_test). Lembre-se que o modelo naive bayes deve usar os dados da vetorização bag-of-words e os demais modelos os dados da vetorização bag-of-words que foram padronizados.

In [ ]:
# Predição do Modelo: Naive Bayes

print(cnb_pred[:10])

In [ ]:
# Predição do Modelo: Regressão Logística

print(clr_pred[:10])

In [ ]:
# Predição do Modelo: KNN

print(knn_pred[:10])

2. **Plote a matriz de confusão para cada modelo.**

Vamos investigar a relação de erros e acertos de cada modelo observando a matriz de confusão. Use as variáveis do passo anterior que armazenam as predições geradas.

In [ ]:
# Faça o plot da matriz de confusão de contagem para cada um dos modelos.
import seaborn as sns

def plot_confusion_matrix(y_true, y_pred, labels=None) -> None:
    cm = confusion_matrix(y_true, y_pred, labels=labels)

    plt.figure(figsize=(len(labels), len(labels)))
    sns.heatmap(
        cm,
        annot=True,
        fmt='d',
        cmap='Blues',
        xticklabels=labels,
        yticklabels=labels,
    )
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()

In [ ]:
# Modelo: Naive Bayes. Dica use: plot_confusion_matrix()

In [ ]:
# Modelo: Regressão Logística. Dica use: plot_confusion_matrix()

In [ ]:
# Modelo: KNN. Dica use: plot_confusion_matrix()

### **Perguntas**

Responda demonstrando o cálculo quando aplicável

1. Qual modelo está sofrendo mais por conta de falsos positivos?
- Resposta:

2. Qual modelo está sofrendo mais por conta de falsos negativos?
- Resposta:

3. Considerando a classe positiva, qual o modelo mais preciso?
- Resposta:

4. Considerando a classe positiva, qual o modelo tem melhor revocação?
- Resposta:


3. **Cálculo de Métricas.**

Vamos calcular e imprimir todas as métricas (acurácia, recall, precision, f1-score). Para isso basta usar a função classification_report() da sklearn.

In [ ]:
# Resultado para o modelo Naive Bayes
print(classification_report(y_test, cnb_pred))

In [ ]:
# Resultado para o modelo de Regressão Logística
print(classification_report(y_test, clr_pred))

In [ ]:
# Resultado para o modelo KNN
print(classification_report(y_test, knn_pred))

### **Perguntas**

Responda Justificando

1. No nosso cenário, é adequado utilizar a métrica de acurácia?
- Resposta:

2. Considerando a classe negativa, qual é o modelo mais preciso?
- Resposta:

3. Considerando a classe negativa, qual o modelo tem melhor revocação?
- Resposta:

## **Interpretando os modelos**

**Uma subárea importante da aprendizagem de máquina é a interpretação dos modelos.**

**Nesta parte do laboratório, você deve utilizará funções já implementadas para facilitar a interpretação dos modelos treinados e responder algumas perguntas sobre eles.**

As funções a seguir utilizam informações das frequências e valores dos pesos dos modelos para encontrar as palavras que mais contribuem para classificação.

O modelo de Regressão Logística tem um parâmetro chamado ***coef_*** este parâmetro retorna o peso de cada feature (palavra) tem no modelo. Palavras com peso positivo influenciam para a classificação positiva, e palavras com peso negativo fazem o inverso. O valor desse parâmetro tem a dimensão (1, n_features).

Já o modelo Naive Bayes tem um parâmetro chamado ***feature_log_prob_***. Este parâmetro retorna o log das probabilidades de cada palavra aparecer no texto dada uma classe. O valor desse parâmetro tem a dimensão (2, n_features), de modo que a posição 0 corresponde as probabilidades para a classe negativa e a posição para a classe positiva. Quanto maior for a probabilidade de uma palavra, maior podemos dizer que é sua influência na classificação.


In [ ]:
def recupera_palavras_positivas(modelo, vectorizer, X_train=None, y_train=None):
    # Verifica o tipo do modelo e obtém os coeficientes
    if isinstance(modelo, SGDClassifier):
        # Implemente aqui usando coef_[0]
    elif isinstance(modelo, MultinomialNB):
        # Implemente aqui usando feature_log_prob_[1]
    elif isinstance(modelo, KNeighborsClassifier):
        # Para KNN, vamos calcular a importância baseada na frequência das palavras nas instâncias positivas do conjunto de treino
        indices_positivos = np.where(y_train == 'pos')[0]

        # Verificar se existem instâncias positivas
        if len(indices_positivos) == 0:
            return "Não há instâncias positivas no conjunto de treino"

        X_positivos = X_train[indices_positivos]

        # Calcula a média das features para instâncias positivas
        coeficientes = np.mean(X_positivos, axis=0)

        # Se for uma matriz esparsa, converte para array
        if hasattr(coeficientes, 'A1'):
            coeficientes = coeficientes.A1
    else:
        coeficientes = modelo.coef_[0]

    # Obter as palavras do vocabulário
    palavras = vectorizer.get_feature_names_out()

    # Ordenar as palavras pelos coeficientes de forma decrescente
    palavras_positivas = np.argsort(coeficientes)[-20:][::-1]

    return [(palavras[i], coeficientes[i]) for i in palavras_positivas]

print("Regressão Logística: " + str(recupera_palavras_positivas(clr, vectorizer)))
print("Naive Bayes: " + str(recupera_palavras_positivas(cnb, vectorizer)))
print("KNN: " + str(recupera_palavras_positivas(knn, vectorizer, X_train, y_train)))

In [ ]:
def recupera_palavras_negativas(modelo, vetorizer, X_train=None, y_train=None):
    # Verifica o tipo do modelo e obtém os coeficientes
    if isinstance(modelo, SGDClassifier):
        # Implemente aqui usando coef_[0]
    elif isinstance(modelo, MultinomialNB):
        # Implemente aqui usando feature_log_prob_[1]
    elif isinstance(modelo, KNeighborsClassifier):
        # Para KNN, vamos calcular a importância baseada na frequência das palavras nas instâncias negativas do conjunto de treino
        indices_negativos = np.where(y_train == 'neg')[0]

        # Verificar se existem instâncias negativas
        if len(indices_negativos) == 0:
            return "Não há instâncias negativas no conjunto de treino"

        X_negativos = X_train[indices_negativos]

        # Calcula a média das features para instâncias negativas
        coeficientes = np.mean(X_negativos, axis=0)

        # Se for uma matriz esparsa, converte para array
        if hasattr(coeficientes, 'A1'):
            coeficientes = coeficientes.A1

        # Filtrar valores NaN e infinitos logo após calcular
        coeficientes = np.nan_to_num(coeficientes, nan=0.0, posinf=0.0, neginf=0.0)
    else:
        coeficientes = modelo.coef_[0]

    # Obter as palavras do vocabulário
    palavras = vetorizer.get_feature_names_out()

    # Para outros modelos, também aplicar filtro NaN
    if not isinstance(modelo, KNeighborsClassifier):
        coeficientes = np.nan_to_num(coeficientes, nan=0.0, posinf=0.0, neginf=0.0)

    # Para KNN, filtrar palavras com coeficiente zero
    if isinstance(modelo, KNeighborsClassifier):
        # Pegar apenas índices onde coeficiente > 0
        indices_nao_zero = np.where(coeficientes > 0)[0]
        if len(indices_nao_zero) == 0:
            return "Não há palavras com coeficientes positivos"

        # Ordenar apenas os coeficientes não-zero de forma crescente
        indices_ordenados = indices_nao_zero[np.argsort(-coeficientes[indices_nao_zero])[:20]]
        return [(palavras[i], coeficientes[i]) for i in indices_ordenados]
    else:
        # Para outros modelos, manter como estava
        palavras_negativas = np.argsort(coeficientes)[:20]
        return [(palavras[i], coeficientes[i]) for i in palavras_negativas]

print("Regressão Logística: " + str(recupera_palavras_negativas(clr, vectorizer)))
print("Naive Bayes: " + str(recupera_palavras_negativas(cnb, vectorizer)))
print("KNN: " + str(recupera_palavras_negativas(knn, vectorizer, X_train, y_train)))

### **Visualizando e interpretando**

Use a função abaixo para visualizar uma nuvem de palavras do retorno das funções.

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

def plot_wordcloud(lista_de_tuplas):
    # Cria um dicionário a partir da lista de tuplas
    palavra_freq = {t[0]: t[1] for t in lista_de_tuplas}

    # Cria a WordCloud
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(palavra_freq)

    # Plota a WordCloud
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.show()

#### **Palavras positivas**

In [ ]:
# Regressão Logística
Y = recupera_palavras_positivas(clr, vectorizer)
plot_wordcloud(Y)

In [ ]:
# Naive Bayes
Y = recupera_palavras_positivas(cnb, vectorizer)
plot_wordcloud(Y)

In [ ]:
# KNN
Y = recupera_palavras_positivas(knn, vectorizer, X_train, y_train)
plot_wordcloud(Y)

#### **Palavras negativas**

In [ ]:
# Regressão Logística
Y = recupera_palavras_negativas(clr, vectorizer)
plot_wordcloud(Y)

In [ ]:
# Naive Bayes
Y = recupera_palavras_negativas(cnb, vectorizer)
plot_wordcloud(Y)

In [ ]:
# KNN
Y = recupera_palavras_negativas(knn, vectorizer, X_train, y_train)
plot_wordcloud(Y)

#### **Perguntas**

**1. Analisando as nuvens de palavras positivas de cada modelo, é possível identificar que as palavras estão associadas à um sentimento positivo? Dê exemplos.**

- Resposta:

**2. Analisando as nuvens de palavras negativas de cada modelo, é possível identificar que as palavras estão associadas à um sentimento negativo? Dê exemplos.**

- Resposta:

